# Model Comparison
---

1. Data Preparation
2. Model Comparison
- K-mean
- LDA
- NMF
- Top2Vec
- BERTopic

## 0. Import Libraries

In [88]:
# Import Basic Libraries
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Import Sklearn Libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans


# Import NLP Libraries
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
from pythainlp.tokenize import sent_tokenize, word_tokenize
from pythainlp.corpus import thai_stopwords
import re
from tqdm import tqdm 
import pyLDAvis
import pyLDAvis.gensim_models
from top2vec import Top2Vec


# Set default Thai font
mpl.font_manager.fontManager.addfont('./THSarabunNew/THSarabunNew.ttf')
mpl.rc('font', family='TH Sarabun New', size=20)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## 1. Data Preparation

### 1.1 Topic Selection

In [11]:
econ_b = pd.read_json('../datasets/เศรษฐกิจ_processed.json')
econ_m = pd.read_json('../datasets/economy_processed.json')

### 1.2 Bag-of-Words

In [12]:
# Finction to store n_word in dict
def featurize(token_list):
    token_list=token_list
    features = {}
    for token in token_list:
        features[token]=1
    return features

In [14]:
econ_b_bow = econ_b['article_tokenize'].apply(featurize)
econ_m_bow = econ_m['article_tokenize'].apply(featurize)

In [23]:
econ_b_bow.shape, econ_m_bow.shape

((6801,), (5884,))

In [89]:
vectorizer_1 = DictVectorizer(sparse=True)
econ_b_vec = vectorizer_1.fit_transform(econ_b_bow)

vectorizer_2 = DictVectorizer(sparse=True)
econ_m_vec = vectorizer_2.fit_transform(econ_m_bow)

In [47]:
econ_b_vec, econ_m_vec

(<6801x35157 sparse matrix of type '<class 'numpy.float64'>'
 	with 844532 stored elements in Compressed Sparse Row format>,
 <5884x40835 sparse matrix of type '<class 'numpy.float64'>'
 	with 891436 stored elements in Compressed Sparse Row format>)

## 2. Model Comparison

### 2.1 K-Means

In [80]:
def kmeans_topic(data_vec=None, n_clusters=10, ):
    kmeans = KMeans(n_clusters=n_clusters, 
                    init='k-means++', 
                    max_iter=100, 
                    n_init=1, 
                    random_state=42)

    kmeans.fit(data_vec)

    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    
    if data_vec == econ_m_vec:
        terms = vectorizer_1.get_feature_names_out()
    else:
        terms = vectorizer_2.get_feature_names_out()

    for i in range(n_clusters):
        print(f'Topic {i+1}')
        print('-'*10)
        for i in order_centroids[i,:10]:
            print(terms[i])
        print('\n')

In [84]:
kmeans_topic(data_vec=econ_b_vec, n_clusters=10)

Topic 1
----------
ช่างคิด
คงกระพัน
ปัตย์
ตากแห้ง
241
without
18.5
สวามิภักดิ์
ฟ็อกซ์
ราม


Topic 2
----------
0
จำยอม
น้ำเน่า
ผู้บรรยาย
onedee
ศิริลักษณ์
ยาแผนโบราณ
ปัตย์
WiFi-CAT
ตื่นตระหนก


Topic 3
----------
กลุ่มตัวอย่าง
ปรึกษาหารือ
รากฐาน
วจา
คงที่
ทางวิชาการ
08
take
ชื่อสกุล
ซึ


Topic 4
----------
ซีคอนสแควร์
ออกดอกออกผล
ดังที่
15.72
14.72
08
ปรึกษาหารือ
241
หมดอายุ
พิมาย


Topic 5
----------
ส์"
ฟ้า
กุมภาพันธ์
ฟ้อ
รายชื่อ
บางบาล
ความต้องการ
อวสาน
things
18.5


Topic 6
----------
การบรรลุผล
ตกรอบ
Utility
ช่างคิด
ผยง
Tile
องศา
พอได้
คั่ว
Muang


Topic 7
----------
สะบักสะบอม
ลงสี
ฉ้อฉล
ภาคอีสาน
กรมปศุสัตว์
อาละวาด
ศิริลักษณ์
คณะที่ปรึกษา
ฉีด
สปริงส์


Topic 8
----------
การบรรทุก
หิน
ที่ทาง
1.9
97.5
ค่าเช่า
Trainer
2025
ATTENTION
molecule


Topic 9
----------
คนใดคนหนึ่ง
ตัด
Tile
ภาวะการณ์
ตากแห้ง
ออกซิเจน
การซ่อมแซม
1,366.59
วาณิชธนกิจ
ภัสร์


Topic 10
----------
บล็อก
ออกดอกออกผล
ขนถ่ายสินค้า
1,366.59
ซีคอนสแควร์
คนใดคนหนึ่ง
การซ่อมแซม
วาณิชธนกิจ
2025
ภัสร์




In [86]:
true_k = 10
kmeans = KMeans(n_clusters=true_k, 
                init='k-means++', 
                max_iter=100, 
                n_init=1, 
                random_state=42)

kmeans.fit(econ_b_vec)

order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer_1.get_feature_names_out()

for i in range(true_k):
    print(f'Topic {i+1}')
    print('-'*10)
    for i in order_centroids[i,:10]:
        print(terms[i])
    print('\n')

Topic 1
----------
ประกาศ
ตาสว่าง
วันที่
พยากรณ์อากาศ
3
คุณหญิง
20
เหมือน
ส่งผล
เกิดขึ้น


Topic 2
----------
0
นโยบาย
รับประทาน
ส
คลอเรสเตอรอล
เม
อฯ
วันที่
การควบคุม
พลิก


Topic 3
----------
จูงใจ
ล้าน
เกณฑ์
เท็จ
ตำบล
มติ
1
คอมฯ
ป.ป.ช.
ปรักปรำ


Topic 4
----------
ประเทศ
ไทย
ปี
2
19
1
ล้าน
3
โควิด
สำหรับ


Topic 5
----------
แสน
ส่งเสริม
ดา
ส่งมอบ
เกินไป
รู้
ทยอย
ได้ผล
คัล
20


Topic 6
----------
ช่วยกัน
ผู้ตอบ
กลไก
ประกาศ
วีซ่า
กรมอุตุนิยมวิทยา
ในขณะเดียวกัน
สัปดาห์
ที่จะ
T


Topic 7
----------
เอ
เจ
บริษัท
ส์
งาน
์
เม
ติก
บริการ
เวน


Topic 8
----------
ช่วงเวลา
โลตัส
มาตรการป้องกัน
13.10
Big
น.
กระทรวง
2563
COVID-
คนใน


Topic 9
----------
ต่อเนื่อง
ฝนตก
กรมอุตุนิยมวิทยา
หนัก
พยากรณ์อากาศ
ไท
ชี้
10
เบื้องหลัง
ส่อง


Topic 10
----------
รายละเอียด
ไทย
ดู
10
ประเทศ
ต่อเนื่อง
ชี้
เบื้องหลัง
2563
ส่อง




### 2.2 LDA

In [91]:
# Function for modeling with LDA
def lda_model(data=None, num_topics=None): #, topicid=None
    
    dictionary = Dictionary(data)
    corpus = [dictionary.doc2bow(txt) for txt in data]
    
    model = LdaModel(corpus=corpus, num_topics=num_topics)
    #topic = pd.DataFrame(model.get_topic_terms(topicid=1, topn=20)).rename(columns={0:'index', 1:'probability'})
    for i in range(num_topics):
        top_n = [dictionary[index] for index, prob in model.get_topic_terms(topicid=i, topn=30)]
        print(f'Topic {i+1}')
        print(top_n)
        print('-'*60)
    return model

In [92]:
# Visualization
def lda_vis(data=None, num_topics=20):
    dictionary = Dictionary(data)
    corpus = [dictionary.doc2bow(txt) for txt in data]
    model = LdaModel(corpus=corpus, num_topics=num_topics)
    pyLDAvis.enable_notebook()
    viz = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    return viz

In [93]:
lda_10b = lda_model(data=econ_b['article_tokenize'], num_topics=10)

Topic 1
['เกษตรกร', 'ไทย', 'สินค้า', '2563', 'ปี', 'เดือน', 'บาท', 'ราคา', 'ยาง', 'ตลาด', 'การส่งออก', 'ประเทศ', 'เพิ่มขึ้น', 'ล้าน', 'ประกัน', '3', 'มาตรการ', 'วันที่', 'การผลิต', 'ปลูก', 'การค้า', '19', 'โครงการ', 'สำหรับ', 'ไฟฟ้า', 'รายได้', '1', 'ยางพารา', 'ผลผลิต', '2']
------------------------------------------------------------
Topic 2
['ราคา', 'สหรัฐฯ', 'เพิ่มขึ้น', 'น้ำมันดิบ', '1', 'ลดลง', 'ปรับตัว', 'เดือน', 'ล้าน', '19', 'โควิด', 'บาร์เรล', 'ดอลลาร์', 'ตลาด', 'น้ำมัน', 'คาด', 'เศรษฐกิจ', 'ไทย', 'ปริมาณ', 'ระดับ', 'ประเทศ', 'บาท', 'สัปดาห์', 'การผลิต', 'ปี', '63', 'ต่อเนื่อง', '2563', 'แพร่ระบาด', 'ลด']
------------------------------------------------------------
Topic 3
['เศรษฐกิจ', 'ประเทศ', 'ไทย', 'ปี', 'โควิด', '19', 'ระบาด', 'ผลกระทบ', 'โลก', 'คน', 'การท่องเที่ยว', '2', 'เรื่อง', 'รัฐบาล', 'สถานการณ์', 'วิกฤติ', 'ทำ', 'นักท่องเที่ยว', '2563', '1', 'พื้นที่', 'ปัญหา', 'มาตรการ', 'ทางเศรษฐกิจ', '3', 'นโยบาย', 'ที่จะ', 'รายได้', 'เกิดขึ้น', 'แพร่ระบาด']
-------------------

In [ ]:
#lda_vis(data=econ_b['article_tokenize'], num_topics=10)
# Topic overlaps

In [ ]:
lda_7b = lda_model(data=econ_b['article_tokenize'], num_topics=7)

In [ ]:
#lda_vis(data=econ_b['article_tokenize'], num_topics=7)
# Topic overlap to the low right

In [ ]:
lda_15b = lda_model(data=econ_b['article_tokenize'], num_topics=15)

In [ ]:
#lda_vis(data=econ_b['article_tokenize'], num_topics=15)
# To the left with one to the right

In [ ]:
lda_30b = lda_model(data=econ_b['article_tokenize'], num_topics=30)

In [ ]:
#lda_vis(data=econ_b['article_tokenize'], num_topics=15)

In [ ]:
lda_10m = lda_model(data=econ_m['article_tokenize'], num_topics=10)

In [ ]:
#overlap กันสูง
#lda_vis(data=econ_m['article_tokenize'], num_topics=10)

In [94]:
lda_7m = lda_model(data=econ_m['article_tokenize'], num_topics=7)

Topic 1
['ไทย', '19', 'สถานการณ์', 'เศรษฐกิจ', 'ธุรกิจ', 'โควิด', 'มาตรการ', 'ประเทศ', 'ปี', '2563', 'วันที่', 'ลูกค้า', 'เรื่อง', 'ทำ', '1', 'หนี้', '2', 'ผลกระทบ', 'สำหรับ', 'ลด', 'กล่าวว่า', 'บริษัท', 'ลูกหนี้', 'หุ้น', 'พนักงาน', 'ธปท.', 'ธนาคาร', '3', 'ล้าน', 'ระดับ']
------------------------------------------------------------
Topic 2
['2', 'รถ', 'บริเวณ', 'ไทย', 'ประเทศ', 'บาท', 'วันที่', 'ราคา', 'รถยนต์', 'อุณหภูมิ', 'ระบบ', 'ฝน', '1', '3', 'องศาเซลเซียส', '2563', 'ทองคำ', 'เมตร', 'สำหรับ', 'พื้นที่', 'คลื่น', 'ปี', '4', 'ยาง', 'จังหวัด', 'รุ่น', 'เวลา', 'ประกาศ', '5', 'แรง']
------------------------------------------------------------
Topic 3
['ไทย', 'ประเทศ', 'ปี', 'วันที่', '2', 'โครงการ', 'บาท', 'ล้าน', 'คน', '1', 'ประชาชน', '2563', 'เศรษฐกิจ', 'บริษัท', '3', 'สถานการณ์', 'Matichon', 'เกาะติด', 'สำหรับ', '5', 'ข้อมูล', 'เรื่อง', 'กล่าวว่า', 'ระบบ', '19', 'เดือน', 'มาตรการ', 'จำนวน', 'รัฐบาล', 'โควิด']
------------------------------------------------------------
Topic 4
['บา

In [113]:
lda_vis(data=econ_m['article_tokenize'], num_topics=7)

/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.071653 -0.029547       1        1  17.672589
1      0.026530  0.020311       2        1  16.982061
2     -0.040947 -0.041296       3        1  16.553446
5      0.073085  0.008323       4        1  13.605763
0     -0.009145  0.027242       5        1  13.286380
6     -0.064367 -0.050059       6        1  12.300214
4     -0.056807  0.065025       7        1   9.599547, topic_info=           Term          Freq         Total Category  logprob  loglift
2029  ลงทะเบียน   2539.000000   2539.000000  Default  30.0000  30.0000
2855      สิทธิ   2746.000000   2746.000000  Default  29.0000  29.0000
8838       คนละ   1692.000000   1692.000000  Default  28.0000  28.0000
1182    โครงการ  10095.000000  10095.000000  Default  27.0000  27.0000
306      ธุรกิจ   6226.000000   6226.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
360         ไทย    636.883138  13847.282274   Topic7  -5.4347  -0.7358
250      สำหรับ    502.075562   5325.481433   Topic7  -5.6726  -0.0181
410      บริษัท    512.405424   7379.170281   Topic7  -5.6522  -0.3238
123        ล้าน    506.119460  10655.926339   Topic7  -5.6646  -0.7036
337      สินค้า    443.016365   4241.853766   Topic7  -5.7977   0.0843

[657 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
19566      3  0.926376     0.09
13930      2  0.029135     0.40
13930      3  0.029135     0.40
13930      5  0.029135     0.40
13930      7  0.903181     0.40
...      ...       ...      ...
4396       5  0.012698   ไฮบริด
4396       6  0.029628   ไฮบริด
4396       7  0.050791   ไฮบริด
13520      1  0.177144  ไฮเปอร์
13520      4  0.806988  ไฮเปอร์

[2512 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 6, 1, 7, 5])

### 2.3 NMF

In [48]:
#data = econ_b['article_tokenize'].apply(lambda x:' '.join(x))
#cvec = CountVectorizer(token_pattern= "\b[A-zก-๙][A-z\.\-ก-๙]*\b")
#data = cvec.fit_transform(data)

In [49]:
nmf = NMF(n_components=10, random_state=42)
nmf.fit(econ_b_vec)

NMF(n_components=10, random_state=42)

In [50]:
nmf_features = nmf.transform(econ_b_vec)
nmf_features.shape

(6801, 10)

In [51]:
nmf.components_.shape

(10, 35157)

In [61]:
econ_b_components = pd.DataFrame(nmf.components_, columns=vectorizer_1.get_feature_names())
econ_b_components 

,0,0.0,0.00,0.001,0.003,0.005,0.0098,0.01,0.010,0.014,...,์เบิต,์เบิร์ก,์เวง,์เฮ้าส์,ํ่า,ํ้า,๒,๒๐๑๙,๒๕๔๑,๒๕๖๓
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.013844,0.000000,0.000000,0.000491,0.000045
1,0.000000,0.000000,0.000000,0.000000,0.001582,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000809,0.000000,0.000000,0.000000,0.000000
2,0.107089,0.005027,0.002787,0.000134,0.000000,0.000134,0.156440,0.025602,0.000000,0.000000,...,0.000928,0.000000,0.000000,0.002389,0.003974,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.031088,0.000000,0.000000,0.000000,0.000108,0.000000,0.000000,0.005034,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000042
4,0.051447,0.000000,0.000000,0.000278,0.000473,0.000278,0.097480,0.023438,0.000862,0.000000,...,0.000000,0.000782,0.001125,0.000000,0.000000,0.000000,0.001757,0.001757,0.000250,0.002077
5,0.055716,0.000000,0.000000,0.003978,0.000000,0.003978,0.000000,0.004295,0.003390,0.003535,...,0.000312,0.000488,0.003625,0.002073,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.012380,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001368,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.004262,0.000691,0.012437,0.000000,0.000000,0.000000,0.063902,0.000000,0.000538,0.001078,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000510,0.000000,0.000000,0.000000,0.000000
8,0.041804,0.000000,0.000000,0.000000,0.000662,0.000000,0.014382,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000489,0.000000,0.000600,0.000220,0.000220,0.000000,0.000076
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [60]:
# Topic & Words
for topic in range(10):
    topic = econ_b_components.iloc[topic]
    print(type(topic))
    print(f'For topic {topic+1} the words with the highest value are:')
    print(topic.nlargest(10))
    print('\n')

<class 'pandas.core.series.Series'>
For topic 0        1.000000
0.0      1.000000
0.00     1.000000
0.001    1.000000
0.003    1.000000
           ...   
ํ้า      1.013844
๒        1.000000
๒๐๑๙     1.000000
๒๕๔๑     1.000491
๒๕๖๓     1.000045
Name: 0, Length: 35157, dtype: float64 the words with the highest value are:
พัฒนา       2.926964
โครงการ     2.557163
การพัฒนา    2.390126
พื้นที่     2.142602
สร้าง       2.046783
ปี          1.981855
การลงทุน    1.735550
ส่งเสริม    1.728226
ระบบ        1.692924
เขต         1.596687
Name: 0, dtype: float64


<class 'pandas.core.series.Series'>
For topic 0        1.000000
0.0      1.000000
0.00     1.000000
0.001    1.000000
0.003    1.001582
           ...   
ํ้า      1.000809
๒        1.000000
๒๐๑๙     1.000000
๒๕๔๑     1.000000
๒๕๖๓     1.000000
Name: 1, Length: 35157, dtype: float64 the words with the highest value are:
กรมอุตุนิยมวิทยา    3.343484
พยากรณ์อากาศ        3.337378
ฝนตก                3.328116
ไท                  3.287378
หนัก  

### 2.4 Top2Vec
- fast learn, learn and deep learn

In [97]:
print(Top2Vec.__doc__)


    Top2Vec

    Creates jointly embedded topic, document and word vectors.


    Parameters
    ----------
    documents: List of str
        Input corpus, should be a list of strings.

    min_count: int (Optional, default 50)
        Ignores all words with total frequency lower than this. For smaller
        corpora a smaller min_count will be necessary.

    ngram_vocab: bool (Optional, default False)
        Add phrases to topic descriptions.

        Uses gensim phrases to find common phrases in the corpus and adds them
        to the vocabulary.

        For more information visit:
        https://radimrehurek.com/gensim/models/phrases.html

    ngram_vocab_args: dict (Optional, default None)
        Pass custom arguments to gensim phrases.

        For more information visit:
        https://radimrehurek.com/gensim/models/phrases.html

    embedding_model: string or callable
        This will determine which model is used to generate the document and
        word embeddings. T

In [ ]:
# min_count =
# ngram_vocab =
# embedding_model= (doc2vec)
# workers = cpu amount

In [102]:
type(econ_b['article_tokenize'])

pandas.core.series.Series

In [106]:
econ_list = econ_b['article'].tolist()
type(econ_list)

list

In [108]:
top2vec = Top2Vec(econ_list, speed='fast-learn')

2022-11-14 08:43:43,610 - top2vec - INFO - Pre-processing documents for training
2022-11-14 08:43:46,943 - top2vec - INFO - Creating joint document/word embedding
2022-11-14 08:43:56,729 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-14 08:44:13,930 - top2vec - INFO - Finding dense areas of documents
2022-11-14 08:44:14,079 - top2vec - INFO - Finding topics


In [109]:
topic_sizes, topic_nums =top2vec.get_topic_sizes()
print(topic_sizes)
print(topic_nums)

[3627  493  277  257  253  234  189  174  168  167  166  149  119  117
  104  102   89   72   44]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]


In [112]:
topic_words, word_scores, topic_nums = top2vec.get_topics(18)
for words, scores, num in zip(topic_words, word_scores, topic_nums):
    print(num)
    print(f'Words: {words}')

0
Words: ['ประจำป' 'เปดเผยวา' 'รวมกบ' 'ในวนท' 'ยนราคาทกโลกรมละ' 'ปลาปนเบอร'
 'เกรดทสงกวา' 'ปลาปน' 'เมลดถวเหลอง' 'เปนตนละ' 'เปนกโลกรมละ' 'โปรตน' 'ทง'
 'บชเชล' 'โรงงานอาหารสตว' 'สวนปลาปนเบอร' 'ชอตตน' 'mail' 'usda' 'โปรตนขนไป'
 'กโลกรมตอตว' 'ชนดทมโปรตนสงกวา' 'สวนปลายขาว' 'เอฟ' 'ลานบชเชล' 'ราคาทรงตว'
 'ถวเหลอง' 'ราคากโลกรมละ' 'ราคาเพมขน' 'แนวโนม' 'ไกเนอ' 'หนาฟารมเกษตรกร'
 'เมอ' 'ดงกลาว' 'โอ' 'พเศษ' 'บวก' 'บรพชยศร' 'ขาวขาว' 'สปดาหน' 'สกร'
 'โดยในป' 'เซนต' 'ขาวโพดเลยงสตว' 'ไปแลว' 'อกครง' 'ประจำวนท' 'ลาสด' 'เอ'
 'รฐตองรบทำ']
1
Words: ['อาเซยน' 'ทรพยทวธรรม' 'โดยในป' 'นางอรมน' 'อนโดนเซย' 'จน' 'นวซแลนด'
 'ประเทศ' 'วอนขอพร' 'เดอนแรกของป' 'อนเดย' 'สนค' 'สหรฐฯ' 'นายกรต' 'เกาหลใต'
 'หดตว' 'ขยายตว' 'นางสาวพมพชนก' 'มาเลเซย' 'กระทรวงพาณชย' 'ลกษณวศษฎ'
 'มมลคา' 'ฉบบ' 'ลานเหรยญสหรฐ' 'เวยดนาม' 'สงคโปร' 'สรท' 'ลานดอลลาร' 'asean'
 'ญปน' 'ออสเตรเลย' 'บาทตอดอลลาร' 'ตดลบ' 'ฮองกง' 'รอบ' 'ครงท' 'นายสมเดจ'
 'มลคา' 'ของโลก' 'ลาว' 'wto' 'การทองเทยว' 'เมยนมา' 'ไตหวน' 'กรมฯ' 'ยโรป'
 'เปดเผยวา' 'พบวา' 'อย' 'ขาว']
2
Wor

In [ ]:
documents, document_scores, document_ids = top2vec.search_documents_by_topic(topic_num=0, 
                                                                             num_docs=10)
for doc, score. doc_id in zip(documents, document_scores, document_ids):
    print(f'Document: {doc_id}, Score: {score}')
    print('-'*20)
    print(doc)
    print('-'*20)
    print()

### 2.5 BERTopic